In [1]:
import wandb
from pprint import pprint

import gym
import d4rl
import pickle
import numpy as np

No module named 'flow'
No module named 'carla'
pybullet build time: Mar  7 2022 18:18:14


## Wandbからrunを取得

In [2]:
path = 'st46/decision-transformer'
api = wandb.Api()
runs = api.runs(
    path=path,
    filters={'state':'finished'}
    )

for run in runs:
    if run.name == 'gym-experiment-hopper-medium-gpt2-666':
        break

## runからreturnを取得

In [36]:
return_map = {}
return_map['medium'] = max(run.history()['evaluation/target_3600_return_mean'])

## RandomとExpertの結果を取得

In [38]:
env_name = "hopper"
datasets = ["random", "expert"]

for dataset in datasets:

    if env_name == "hopper":
        env = gym.make("Hopper-v3")
        max_ep_len = 1000
        env_targets = [3600, 1800]  # evaluation conditioning targets
        scale = 1000.0  # normalization for rewards/returns
    elif env_name == "halfcheetah":
        env = gym.make("HalfCheetah-v3")
        max_ep_len = 1000
        env_targets = [12000, 6000]
        scale = 1000.0
    elif env_name == "walker2d":
        env = gym.make("Walker2d-v3")
        max_ep_len = 1000
        env_targets = [5000, 2500]
        scale = 1000.0
    elif env_name == "reacher2d":
        from decision_transformer.envs.reacher_2d import Reacher2dEnv

        env = Reacher2dEnv()
        max_ep_len = 100
        env_targets = [76, 40]
        scale = 10.0
    else:
        raise NotImplementedError

    state_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]

    # load dataset
    dataset_path = f"../data/{env_name}-{dataset}-v2.pkl"
    with open(dataset_path, "rb") as f:
        trajectories = pickle.load(f)

    # save all path information into separate lists
    mode = "normal"
    states, traj_lens, returns = [], [], []
    for path in trajectories:
        if mode == "delayed":  # delayed: all rewards moved to end of trajectory
            path["rewards"][-1] = path["rewards"].sum()
            path["rewards"][:-1] = 0.0
        states.append(path["observations"])
        traj_lens.append(len(path["observations"]))
        returns.append(path["rewards"].sum())
    traj_lens, returns = np.array(traj_lens), np.array(returns)

    # used for input normalization
    states = np.concatenate(states, axis=0)
    state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6

    num_timesteps = sum(traj_lens)

    print("=" * 50)
    print(f"Starting new experiment: {env_name} {dataset}")
    print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
    print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
    print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
    print("=" * 50)
    
    if dataset == 'random':
        return_map['random'] = np.mean(returns)
    elif dataset == 'expert':
        return_map['expert'] = np.mean(returns)

Starting new experiment: hopper random
45239 trajectories, 999996 timesteps found
Average return: 18.40, std: 17.45
Max return: 292.55, min: 2.93
Starting new experiment: hopper expert
1027 trajectories, 999494 timesteps found
Average return: 3511.36, std: 328.59
Max return: 3759.08, min: 1645.28


## 結果の表示

In [42]:
pprint(return_map)
normalized_score = 100 * (return_map['medium'] - return_map['random']) / (return_map['expert'] - return_map['random'])
print(f'Normalized Score: {normalized_score}')

{'expert': 3511.3577, 'medium': 2839.0381429314557, 'random': 18.398905}
Normalized Score: 80.75214876262116
